In [10]:
import numpy as np
from PIL import Image
from skimage.transform import resize
import matplotlib.pyplot as plt
import cv2


def preprocess_image(image, target_size=(28, 28)):
    """Preprocess the input image to match MNIST data format."""
    # Convert to grayscale
    image = image.convert('L')
    # Resize image to match model's expected input
    #image = resize(np.array(image), target_size, anti_aliasing=True)
    # Normalize pixel values
    image = image / 255.0
    # Reshape for the model
    image = image.reshape(1, 28*28)
    return image

def crop_to_digit_preprocess(image_path):
    # Read the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian Blurring
    blurred = cv2.GaussianBlur(img, (5, 5), 0)

    # Apply Thresholding or Edge Detection
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the digit
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop the image around the bounding rectangle of the digit
        digit = img[y:y+h, x:x+w]

        # Resize the cropped digit to 28x28 pixels
        digit_resized = cv2.resize(digit, (28, 28), interpolation=cv2.INTER_AREA)

        # Normalize pixel values to be in the range [0, 1]
        digit_normalized = digit_resized / 255.0

        # Ensure the digit is in the correct shape (1, 28, 28) for MNIST-like format
        digit_reshaped = digit_normalized.reshape(1, 28, 28)
        return digit_reshaped
    else:
        return None  # No digit found

In [11]:
image_path = '/home/thor_01/Documents/EC/ensemble-learning-sklearn/notebooks/IMG_2127.jpg'
#image = Image.open(image_path)
digit_image = crop_to_digit(image_path)
#sample = preprocess_image(digit_image)

AttributeError: 'numpy.ndarray' object has no attribute 'convert'

In [8]:
digit_image

array([[220, 217, 219, ..., 204, 204, 203],
       [221, 220, 220, ..., 206, 204, 203],
       [221, 221, 221, ..., 206, 204, 203],
       ...,
       [221, 222, 224, ..., 210, 206, 204],
       [217, 224, 226, ..., 209, 205, 202],
       [218, 220, 221, ..., 207, 203, 200]], dtype=uint8)

In [12]:
plt.imshow(digit_image.reshape(28, 28), interpolation="gaussian")

ValueError: cannot reshape array of size 199356 into shape (28,28)

In [4]:
import cv2
import numpy as np

def crop_to_digit(image_path):
    # Read the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian Blurring
    blurred = cv2.GaussianBlur(img, (5, 5), 0)

    # Apply Thresholding or Edge Detection
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the digit
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop and return the image around the bounding rectangle of the digit
        digit = img[y:y+h, x:x+w]
        return digit
    else:
        return None  # No digit found

# Example usage
digit_image = crop_to_digit(image_path)

if digit_image is not None:
    cv2.imshow("Cropped Digit", digit_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()